In [1]:
#Import necessary libraries
import numpy as np #This works with numbers
import pandas as pd #This allows us to work with a dataset
import matplotlib.pyplot as plt #This allows us to plot data
from sklearn.model_selection import train_test_split #For splitting data for training and test
from sklearn.preprocessing import StandardScaler #For scaling features
from sklearn import datasets, svm 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score #For metrics and evaluation algorithm
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier #For KNN
from sklearn.preprocessing import KBinsDiscretizer
from sklearn import tree

In [2]:
#load data into dataframe
data = pd.read_csv('HCV-Egy-Data.csv')

# Remove Outliers 
X = data
col = list(X.drop('Baselinehistological staging',axis=1).columns)
for i in col:
    y = data[i]
    removed_outliers = y.between(y.quantile(.02), y.quantile(.98))
    index_names = data[~removed_outliers].index 
    data.drop(index_names, inplace=True)
    
#dropping histoligical baseline due to duplicate representation of data
data = data.drop('Baseline histological Grading', axis = 1)
    
#seperate X and y
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

#Calculate biomarker scores
fib4 = (X['Age']*X['AST1'])/(X['Plat']*np.sqrt(X['ALT1']))
modapri = (X['Age']*X['AST1'])/(X['Plat']*100)
liverDam = X['AST1']/X['ALT1']

#Feature selection Age, Gender, BMI
ModX = X.iloc[:,0:3]

#Select ALT scores over time
ModX['ALT1'] = X['ALT1']
ModX['ALT4'] = X['ALT4']
ModX['ALT 24'] = X['ALT 24']
ModX['ALT 48'] = X['ALT 48']

ModX['AST1'] = X['AST1']

ModX['fib4'] = fib4
ModX['modapri'] = modapri
ModX['liverDam'] = liverDam

ModX

<ipython-input-2-468b208819d2>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ModX['ALT1'] = X['ALT1']
<ipython-input-2-468b208819d2>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ModX['ALT4'] = X['ALT4']
<ipython-input-2-468b208819d2>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

,Age,Gender,BMI,ALT1,ALT4,ALT 24,ALT 48,AST1,fib4,modapri,liverDam
3,49,2,33,64,109,88,77,43,0.001798,0.000144,0.671875
5,58,2,22,104,121,65,114,66,0.002860,0.000292,0.634615
6,42,2,26,57,113,107,80,78,0.002448,0.000185,1.368421
7,48,2,30,112,80,45,53,119,0.002497,0.000264,1.062500
9,45,1,30,68,72,81,43,55,0.003056,0.000252,0.808824
...,...,...,...,...,...,...,...,...,...,...,...
1369,32,1,26,98,102,86,110,77,0.001777,0.000176,0.785714
1373,46,1,27,80,64,92,79,118,0.005642,0.000505,1.475000
1376,46,1,27,49,77,103,120,40,0.002176,0.000152,0.816327
1377,52,2,25,50,98,50,122,109,0.004379,0.000310,2.180000


In [3]:
#Split data into training and test data
X_train, X_test, y_train, y_test = train_test_split(ModX, y, test_size = 0.20, random_state = 0)

#Feature scale so that one feature doesn't have more influence than another
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
#Here we are going to make a simple SVM classifier
classifier = svm.NuSVC(nu = 0.3, kernel='rbf')
#Carry out cross validation 10 fold and find the mean of the score to get overall accuracy for train data
scores = cross_val_score(classifier, X_train, y_train, cv=10)
print(scores)
print(scores.mean())

[0.30434783 0.1884058  0.14492754 0.26086957 0.25       0.27941176
 0.23529412 0.26470588 0.19117647 0.27941176]
0.23985507246376808


In [14]:
#train the classifier on train set and then print out its overall accuracy for the train data
classifier.fit(X_train, y_train)

print(classifier.score(X_train,y_train))

0.9766081871345029


In [15]:
#Determine the overall accuracy score of the test data set
print(classifier.score(X_test,y_test))

0.3742690058479532


In [16]:
#Making predictions
y_pred = classifier.predict(X_test)
#To evaluate algorithm we will print the confusion matrix and other metrics
print("The confusion matrix is:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

The confusion matrix is:
[[19 10  9  9]
 [ 7 10  9  6]
 [11  6 24 11]
 [14 12  3 11]]
              precision    recall  f1-score   support

           1       0.37      0.40      0.39        47
           2       0.26      0.31      0.29        32
           3       0.53      0.46      0.49        52
           4       0.30      0.28      0.29        40

    accuracy                           0.37       171
   macro avg       0.37      0.36      0.36       171
weighted avg       0.38      0.37      0.38       171



In [17]:
#Training with k-nearest neighbor (KNN)
classifier2 = KNeighborsClassifier(n_neighbors = 100)
classifier2.fit(X_train, y_train)
#Making predictions
y_pred = classifier2.predict(X_test)
#To evaluate algorithm we will print the confusion matrix and other metrics
print("The confusion matrix is:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

The confusion matrix is:
[[ 8  7  7 25]
 [ 4  6  5 17]
 [ 9 13  9 21]
 [ 3 11  7 19]]
              precision    recall  f1-score   support

           1       0.33      0.17      0.23        47
           2       0.16      0.19      0.17        32
           3       0.32      0.17      0.23        52
           4       0.23      0.47      0.31        40

    accuracy                           0.25       171
   macro avg       0.26      0.25      0.23       171
weighted avg       0.27      0.25      0.24       171



In [18]:
#Decision trees
#Here we are going to make a simple decision classifier
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
print(clf.score(X_train,y_train))
#Determine the overall accuracy score of the test data set
print(clf.score(X_test,y_test))
#Making predictions
y_pred = clf.predict(X_test)
#To evaluate algorithm we will print the confusion matrix and other metrics
print("The confusion matrix is:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

1.0
0.26900584795321636
The confusion matrix is:
[[14 12 12  9]
 [ 6  9  7 10]
 [ 8 15 12 17]
 [12  7 10 11]]
              precision    recall  f1-score   support

           1       0.35      0.30      0.32        47
           2       0.21      0.28      0.24        32
           3       0.29      0.23      0.26        52
           4       0.23      0.28      0.25        40

    accuracy                           0.27       171
   macro avg       0.27      0.27      0.27       171
weighted avg       0.28      0.27      0.27       171

